# Install library

In [ ]:
%pip install roboflow
%pip install ultralytics

# Get dataset

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

ROBOFLOW_API_KEY = os.environ.get("ROBOFLOW_API_KEY")

if not ROBOFLOW_API_KEY:
  raise ValueError("ROBOFLOW_API_KEY not found in environment variables. Please check your .env file.")

In [2]:
from roboflow import Roboflow

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace("deeprod").project("tollway-detection")
version = project.version(1)
dataset = version.download("yolov12")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to tollway-detection-1 in yolov12:: 100%|██████████| 2090/2090 [00:02<00:00, 824.07it/s]


# Train model

In [3]:
!yolo task=detect mode=train model=yolo12n.pt data={dataset.location}/data.yaml epochs=100 imgsz=640

^C


New https://pypi.org/project/ultralytics/8.3.88 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.85 🚀 Python-3.12.3 torch-2.6.0+cpu CPU (AMD Ryzen 7 4800H with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolo12n.pt, data=c:\trollway-detection\training\tollway-detection-1/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_


  0%|          | 0.00/5.34M [00:00<?, ?B/s]
 23%|██▎       | 1.25M/5.34M [00:00<00:00, 12.3MB/s]
 47%|████▋     | 2.50M/5.34M [00:00<00:00, 12.5MB/s]
 70%|███████   | 3.75M/5.34M [00:00<00:00, 12.6MB/s]
 94%|█████████▎| 5.00M/5.34M [00:00<00:00, 12.6MB/s]
100%|██████████| 5.34M/5.34M [00:00<00:00, 12.6MB/s]

train: Scanning C:\trollway-detection\training\tollway-detection-1\train\labels...:   0%|          | 0/876 [00:00<?, ?it/s]
train: Scanning C:\trollway-detection\training\tollway-detection-1\train\labels... 9 images, 0 backgrounds, 0 corrupt:   1%|          | 9/876 [00:00<00:10, 79.21it/s]
train: Scanning C:\trollway-detection\training\tollway-detection-1\train\labels... 32 images, 0 backgrounds, 0 corrupt:   4%|▎         | 32/876 [00:00<00:05, 158.53it/s]
train: Scanning C:\trollway-detection\training\tollway-detection-1\train\labels... 59 images, 0 backgrounds, 0 corrupt:   7%|▋         | 59/876 [00:00<00:04, 203.60it/s]
train: Scanning C:\trollway-detection\training\tollway-det

# Test model

In [13]:
from ultralytics import YOLO
import seaborn as sns
import numpy as np

# Load the model
model = YOLO("../models/vehicle_tracker_model.pt")

# Run evaluation on the validation set
results = model.val(data=f"{dataset.location}/data.yaml")

# Display evaluation metrics
print(f"mAP50: {results.box.map50:.4f}")
print(f"mAP50-95: {results.box.map:.4f}")

Ultralytics 8.3.85  Python-3.12.3 torch-2.6.0+cpu CPU (AMD Ryzen 7 4800H with Radeon Graphics)
YOLOv12n summary (fused): 159 layers, 2,557,508 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\trollway-detection\training\tollway-detection-1\test\labels... 91 images, 0 backgrounds, 0 corrupt: 100%|██████████| 91/91 [00:00<00:00, 247.47it/s]

val: New cache created: C:\trollway-detection\training\tollway-detection-1\test\labels.cache
WARNING  Box and segment counts should be equal, but got len(segments) = 7, len(boxes) = 792. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:11<00:00,  1.96s/it]


                   all         91        792      0.451      0.519      0.422      0.145
               2-Wheel         74        136      0.601      0.632      0.557      0.152
               4-Wheel         91        639      0.685       0.69      0.675       0.26
               6-wheel         15         17     0.0661      0.235     0.0335     0.0215
Speed: 1.8ms preprocess, 114.3ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs\detect\val4
mAP50: 0.4220
mAP50-95: 0.1445
